# Coleta e Preparação de Dados

## Dados de Opções

In [9]:
import yfinance as yf
import pandas as pd
import numpy as np

ticker = 'IBIT'

data = yf.Ticker(ticker)

opt_date = data.options
opt_date

('2024-12-20',
 '2025-01-17',
 '2025-02-21',
 '2025-05-16',
 '2026-01-16',
 '2027-01-15')

In [10]:
df = pd.DataFrame()

for date in opt_date:
    opt_calls = data.option_chain(date).calls
    opt_calls["Expiration_date"] = date
    df = pd.concat([df, opt_calls])
df.reset_index(drop=True, inplace=True)
df

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,Expiration_date
0,IBIT241220C00030000,2024-11-22 20:58:37+00:00,30.0,26.70,26.6,26.9,0.570002,2.181407,170.0,624.0,1.351566,True,REGULAR,USD,2024-12-20
1,IBIT241220C00036000,2024-11-21 14:34:42+00:00,36.0,19.40,20.7,20.9,0.000000,0.000000,2.0,5.0,1.046880,True,REGULAR,USD,2024-12-20
2,IBIT241220C00037000,2024-11-19 18:49:06+00:00,37.0,17.00,0.0,0.0,0.000000,0.000000,NaN,NaN,0.000010,True,REGULAR,USD,2024-12-20
3,IBIT241220C00038000,2024-11-21 17:45:27+00:00,38.0,18.20,18.7,19.0,0.000000,0.000000,4.0,10.0,0.973633,True,REGULAR,USD,2024-12-20
4,IBIT241220C00040000,2024-11-22 20:42:42+00:00,40.0,16.82,16.8,17.0,-0.030001,-0.178046,47.0,152.0,0.897462,True,REGULAR,USD,2024-12-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,IBIT270115C00080000,2024-11-22 20:47:32+00:00,80.0,22.40,21.6,22.5,0.600000,2.752295,60.0,574.0,0.879213,False,REGULAR,USD,2027-01-15
248,IBIT270115C00085000,2024-11-22 16:44:42+00:00,85.0,20.60,20.5,21.7,0.600000,3.000002,3.0,27.0,0.876710,False,REGULAR,USD,2027-01-15
249,IBIT270115C00090000,2024-11-22 19:30:16+00:00,90.0,20.60,19.9,21.0,0.900000,4.568526,63.0,192.0,0.881898,False,REGULAR,USD,2027-01-15
250,IBIT270115C00095000,2024-11-22 19:55:12+00:00,95.0,20.45,19.3,20.4,1.900001,10.242597,22.0,70.0,0.886903,False,REGULAR,USD,2027-01-15


In [11]:
df.to_csv('./data/options.csv', index=False)

## Dados de cotações Indice Bitcoin

In [12]:
import yfinance as yf
import datetime as dt
import pandas as pd
import numpy as np
import requests

start = dt.datetime(2020, 1, 1)
end = dt.datetime.today()

dados = yf.download(ticker, start=start, end=end)
dados

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-01-11,27.940001,30.000000,26.280001,26.629999,26.629999,37657300
2024-01-12,26.400000,26.410000,24.750000,24.969999,24.969999,22846800
2024-01-16,24.700001,24.862000,24.040001,24.719999,24.719999,15078200
2024-01-17,24.350000,24.559999,24.150000,24.410000,24.410000,13964500
2024-01-18,24.350000,24.469999,23.200001,23.340000,23.340000,17823200
...,...,...,...,...,...,...
2024-11-18,51.500000,52.805000,51.040001,52.130001,52.130001,61429300
2024-11-19,52.160000,53.660000,51.910000,52.700001,52.700001,80884000
2024-11-20,53.880001,54.169998,53.060001,53.720001,53.720001,70154500


In [13]:
dados.to_csv('./data/ibit-usd.csv')

In [16]:
dados = pd.read_csv('./data/ibit-usd.csv')

dados['Returns'] = np.log(dados['Adj Close']/dados['Adj Close'].shift(1))
dados = dados[['Date', 'Returns']].dropna()

mi = dados['Returns'].mean()
sigma = dados['Returns'].std()

r = (1 + 4.41/100)**(1/252) - 1

parameters = pd.DataFrame([[mi, sigma, r]], columns=['mi', 'sigma', 'rf']).to_csv('./data/parameters_ibit.csv', index=False)
mi, sigma, r

(0.003433905446294751, 0.03601163174217144, 0.00017126573697634484)